# tf.Variable vs tf.get_variable

https://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow

I'd recommend to always use tf.get_variable(...) -- it will make it way easier to refactor your code if you need to share variables at any time, e.g. in a multi-gpu setting (see the multi-gpu CIFAR example). There is no downside to it.

I can find two main differences between one and the other:

1. First is that tf.Variable will always create a new variable, whether tf.get_variable gets from the graph an existing variable with those parameters, and if it does not exists, it creates a new one.

2. tf.Variable requires that an initial value be specified.

In [1]:
import tensorflow as tf

a = tf.Variable(1., name="v", expected_shape=[1]) 
b = tf.Variable(1., name="v", expected_shape=[1])

assert(a is not b)  #Assertion is true, they are different objects

print(a.name) # v:0
print(b.name) # v_1:0

print(hex(id(a)))
print(hex(id(b)))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    print(sess.run([a, b]))

v:0
v_1:0
0xb3296d748
0xb3296def0
[1.0, 1.0]


In [1]:
import tensorflow as tf

a = tf.get_variable("v", shape=[1]) # v:0 
b = tf.get_variable("v", shape=[1])

# ValueError: Variable v already exists, disallowed. 
# Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? 

ValueError: Variable v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-1-b23888bc8b63>", line 3, in <module>
    a = tf.get_variable("v", shape=[1]) # v:0
  File "/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2901, in run_ast_nodes
    if self.run_code(code, result):


In [1]:
import tensorflow as tf

a = tf.get_variable("v", shape=[1]) 
b = tf.get_variable("v", shape=[1], reuse=True)

# TypeError: get_variable() got an unexpected keyword argument 'reuse'

TypeError: get_variable() got an unexpected keyword argument 'reuse'

In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) #a.name == "one/v:0"
    
with tf.variable_scope("one"):
    b = tf.get_variable("v", shape=[1]) #b.name == "one/v:0"

# ValueError: Variable one/v already exists, disallowed. 
# Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? 

ValueError: Variable one/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-1-1452c42fae41>", line 4, in <module>
    a = tf.get_variable("v", shape=[1]) #a.name == "one/v:0"
  File "/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2901, in run_ast_nodes
    if self.run_code(code, result):


In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) #a.name == "one/v:0"
    
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", shape=[1]) #b.name == "one/v:0"

assert(a is b)  #Assertion is true, they refer to the same object.

print(a.name)
print(b.name)

print(hex(id(a)))
print(hex(id(b)))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    print(sess.run([a, b]))

one/v:0
one/v:0
0xb34054358
0xb34054358
[array([-0.667483], dtype=float32), array([-0.667483], dtype=float32)]


In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #a.name == "one/v:0"
    
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #b.name == "one/v:0"

assert(a is b)  #Assertion is true, they refer to the same object.

print(a.name)
print(b.name)

print(hex(id(a)))
print(hex(id(b)))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    print(sess.run([a, b]))

one/v:0
one/v:0
0xb3283a550
0xb3283a550
[array([1.], dtype=float32), array([1.], dtype=float32)]


In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #a.name == "one/v:0"
    
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #b.name == "one/v:0"

# It is important to clarify that 
# the function tf.get_variable prefixes the name with the current variable scope 
# to perform reuse checks.
with tf.variable_scope("two"):
    c = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #c.name == "two/v:0"

assert(a is b)  #Assertion is true, they refer to the same object.
assert(a is not c)  #Assertion is true, they are different objects

print(hex(id(a)))
print(hex(id(b)))
print(hex(id(c)))

0xb33973588
0xb33973588
0xb33973668


In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #a.name == "one/v:0"
    
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #b.name == "one/v:0"

# The last assertion error is interesting: 
# Two variables with the same name under the same scope 
# are supposed to be the same variable. 
# But if you test the names of variables c and d 
# you will realize that Tensorflow changed the name of variable d:
with tf.variable_scope("two"):
    c = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #c.name == "two/v:0"
    d = tf.Variable(1., name="v", expected_shape=[1]) #d.name == "two/v_1:0"

assert(c is not d)  #Assertion is true, they are different objects

print(c.name)
print(d.name)

print(hex(id(c)))
print(hex(id(d)))

two/v:0
two/v_1:0
0xb33971860
0xb339716a0


In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #a.name == "one/v:0"
    
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #b.name == "one/v:0"

# The last assertion error is interesting: 
# Two variables with the same name under the same scope 
# are supposed to be the same variable. 
# But if you test the names of variables c and d 
# you will realize that Tensorflow changed the name of variable d:
with tf.variable_scope("two"):
    c = tf.Variable(1., name="v", expected_shape=[1]) #c.name == "two/v:0"
    d = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #d.name == "two/v_1:0"

assert(c is not d)  #Assertion is true, they are different objects

print(c.name)
print(d.name)

print(hex(id(c)))
print(hex(id(d)))

two/v:0
two/v_1:0
0xb331705c0
0xb33170898


In [1]:
import tensorflow as tf

with tf.variable_scope("one"):
    a = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #a.name == "one/v:0"
    
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #b.name == "one/v:0"

# The last assertion error is interesting: 
# Two variables with the same name under the same scope 
# are supposed to be the same variable. 
# But if you test the names of variables c and d 
# you will realize that Tensorflow changed the name of variable d:
with tf.variable_scope("two"):
    c = tf.Variable(1., name="v", expected_shape=[1]) #c.name == "two/v:0"
    d = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) #d.name == "two_1/v:0"
    e = tf.get_variable("v", \
                        shape=[1], \
                        initializer=tf.constant_initializer(1.)) 

# ValueError: Variable two/v already exists, disallowed. 
# Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?

ValueError: Variable two/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-1-a121cb690284>", line 16, in <module>
    d = tf.get_variable("v",                         shape=[1],                         initializer=tf.constant_initializer(1.)) #d.name == "two_1/v:0"
  File "/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2901, in run_ast_nodes
    if self.run_code(code, result):
